In [1]:
!rm -rf Eunomia
!git clone https://github.com/ouaguenouni/Eunomia
!pip install -r Eunomia/requirements.txt

Cloning into 'Eunomia'...
remote: Enumerating objects: 532, done.
remote: Counting objects: 100% (532/532), done.
remote: Compressing objects: 100% (204/204), done.
remote: Total 532 (delta 396), reused 458 (delta 322), pack-reused 0
Receiving objects: 100% (532/532), 3.49 MiB | 2.28 MiB/s, done.
Resolving deltas: 100% (396/396), done.


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from Eunomia.preferences import *
from Eunomia.additive_functions import *
from Eunomia.alternatives import *
from Eunomia.sampling import *
from Eunomia.mcmc import *
from Eunomia.degree import *
from Eunomia.experiments import *


In [6]:
import time

TEST_NAME = "SX_IJCAI_M"
REPLACE = False

n = 5
sigma_weights = 1
sigma_w_vals = np.linspace(1e-3, 1e1, 25)
sigma_p_vals =  np.linspace(1e-3, 1e1, 25)
n_samples = 500
k=2

for iter_ in range(10):
    for sigma_w, sigma_p in zip([1], [1]):
        for m in list(i for i in range(2, 2**n, 1)):
            run_d = {
                "iter":iter_,
                "n": n,
                "k": k,
                "m": m,
                "sigma_w": float(sigma_w),
                "sigma_p": float(sigma_p),
                "n_samples": n_samples
            }
            found = find_experiment_file(run_d, TEST_NAME)
            if found:
                date = found
                print(f"Found file: {file_name}")
                if REPLACE:
                    print('Replacing it...')
                else:
                    continue
            else:
                file_name = compute_experiment_file_name(run_d, TEST_NAME)
                print("Registering the results in: ", file_name)
            ####
            theta = generate_additive_theta(n, k)
            weights = generate_normal_weights(theta, sigma_weights)
            alternatives = generate_random_alternatives_matrix(m, n)
            ranks = compute_ws_ranks(alternatives, theta, weights)
            t_sv = compute_semivalues(n, theta, weights, lambda x:1)
            preferences = PreferenceModel(alternatives, ranks)
            data = preferences.generate_preference_matrix(theta)
            data = torch.tensor(data).float()
            t = time.time()
            model = posterior_sampling_model(data, sigma_w=sigma_w, sigma_p=sigma_p)
            diag, sampled_weights, sigmas = sample_model(model, data, "w", "sigma", warmup_steps=500, num_samples=n_samples, return_diag=True)
            t = time.time() - t
            accs_d = get_acc_distribution(data, sampled_weights, sigmas)
            predicted_rankings = [np.argsort(compute_semivalues(n, theta, weights, lambda x:1))[::-1] for weights in sampled_weights]
            kt_d = get_kt_distribution(predicted_rankings, np.argsort(t_sv))
            ###
            
            file_path = record_experiment_results(run_d, TEST_NAME)
            
            run_d["time"] = t
            run_d["weights"] = weights.tolist()
            run_d["predicted_rankings"] = [i.tolist() for i in predicted_rankings]
            run_d["accuracy_distribution"] = [i.tolist() for i in accs_d]
            run_d["kt_d"] = [i.tolist() for i in kt_d]
            run_d["acceptance_rate"] = diag["acceptance rate"]["chain 0"]
            run_d["w_eff"] = [float(i) for i in diag["w"]["n_eff"].numpy()]
            run_d["w_rhat"] = [float(i) for i in diag["w"]["r_hat"].numpy()]
            
            print("diag: ", diag)
            with open(file_path, 'w') as file:
                yaml.dump(run_d, file, default_flow_style=False)


Registering the results in:  SX_IJCAI_M/2024-01-10/iter-0_n-5_k-2_m-2_sigma_w-1.0_sigma_p-1.0_n_samples-500.yaml


Sample: 100%|████████████████████████████| 1000/1000 [00:40, 24.45it/s, step size=3.35e-01, acc. prob=0.819]


diag:  {'sigma': OrderedDict([('n_eff', tensor(555.7994)), ('r_hat', tensor(0.9984))]), 'w': OrderedDict([('n_eff', tensor([536.5211, 553.8564, 532.0381, 347.8598, 686.3341, 618.1019, 566.5269,
        544.1819, 497.0947, 756.5322, 629.0866, 528.3118, 684.3365, 651.7744,
        532.2642])), ('r_hat', tensor([0.9987, 0.9987, 0.9981, 0.9988, 1.0064, 0.9982, 1.0009, 0.9994, 1.0017,
        1.0021, 0.9983, 0.9982, 1.0012, 0.9980, 0.9982]))]), 'divergences': {'chain 0': [30, 44, 89, 159, 178, 187, 191, 257, 370, 385]}, 'acceptance rate': {'chain 0': 0.994}}
Registering the results in:  SX_IJCAI_M/2024-01-10/iter-0_n-5_k-2_m-3_sigma_w-1.0_sigma_p-1.0_n_samples-500.yaml


Sample: 100%|████████████████████████████| 1000/1000 [00:57, 17.41it/s, step size=2.47e-01, acc. prob=0.793]


diag:  {'sigma': OrderedDict([('n_eff', tensor(347.4546)), ('r_hat', tensor(1.0011))]), 'w': OrderedDict([('n_eff', tensor([447.0719, 380.9706, 357.3636, 359.6635, 352.0364, 392.0301, 385.4454,
        376.0336, 343.5939, 376.1972, 330.7195, 494.1231, 515.3904, 340.3011,
        491.1231])), ('r_hat', tensor([1.0019, 0.9980, 0.9985, 1.0003, 0.9987, 0.9984, 0.9982, 1.0012, 0.9980,
        0.9983, 0.9983, 0.9984, 1.0021, 0.9986, 1.0033]))]), 'divergences': {'chain 0': [52, 54, 245, 388, 431]}, 'acceptance rate': {'chain 0': 0.984}}
Registering the results in:  SX_IJCAI_M/2024-01-10/iter-0_n-5_k-2_m-4_sigma_w-1.0_sigma_p-1.0_n_samples-500.yaml


...

diag:  {'sigma': OrderedDict([('n_eff', tensor(77.1480)), ('r_hat', tensor(0.9992))]), 'w': OrderedDict([('n_eff', tensor([ 70.9384,  81.7466,  80.1361,  94.9965,  83.4637, 109.4930,  59.6077,
         61.7645, 104.2496, 109.3913,  61.1363,  61.4919,  59.6842, 139.7155,
         59.2804])), ('r_hat', tensor([0.9999, 1.0051, 1.0056, 0.9990, 1.0004, 0.9985, 0.9982, 1.0000, 1.0005,
        0.9980, 0.9987, 1.0075, 1.0045, 0.9991, 0.9986]))]), 'divergences': {'chain 0': [150, 165, 445, 454]}, 'acceptance rate': {'chain 0': 0.986}}
Registering the results in:  SX_IJCAI_M/2024-01-11/iter-9_n-5_k-2_m-30_sigma_w-1.0_sigma_p-1.0_n_samples-500.yaml


Sample: 100%|████████████████████████████| 1000/1000 [01:35, 10.50it/s, step size=6.39e-02, acc. prob=0.602]


diag:  {'sigma': OrderedDict([('n_eff', tensor(28.0725)), ('r_hat', tensor(0.9998))]), 'w': OrderedDict([('n_eff', tensor([33.9384, 23.4947, 24.5053, 31.2950, 19.6168, 37.9207, 97.9975, 53.4489,
        21.8863, 28.4518, 33.7703, 38.7008, 27.4975, 25.6221, 27.6981])), ('r_hat', tensor([1.0000, 1.0214, 1.0169, 1.0098, 1.0000, 1.0168, 1.0188, 0.9987, 1.0178,
        1.0109, 1.0143, 1.0777, 1.0589, 1.0284, 1.0571]))]), 'divergences': {'chain 0': [88, 155, 192, 200, 202, 206, 210, 220, 221, 223, 234, 235, 237, 238, 239, 271, 323, 328, 337, 338, 340, 341, 344, 345, 347, 352, 353, 355, 359, 416, 419, 420, 423, 424, 431, 435, 437, 438, 439, 440, 441, 445, 449]}, 'acceptance rate': {'chain 0': 0.894}}
Registering the results in:  SX_IJCAI_M/2024-01-11/iter-9_n-5_k-2_m-31_sigma_w-1.0_sigma_p-1.0_n_samples-500.yaml


Sample: 100%|████████████████████████████| 1000/1000 [02:08,  7.78it/s, step size=4.72e-02, acc. prob=0.672]


diag:  {'sigma': OrderedDict([('n_eff', tensor(36.0528)), ('r_hat', tensor(1.0652))]), 'w': OrderedDict([('n_eff', tensor([101.7566,  24.9560,  81.6162,  25.6054, 109.4676,  25.2702, 100.7412,
         26.3813,  41.3086,  31.9919,  30.3724,  42.9845,  21.7805,  49.2399,
         24.0477])), ('r_hat', tensor([0.9983, 1.0042, 1.0039, 1.0041, 0.9982, 1.0051, 1.0047, 1.0078, 1.0083,
        1.0146, 1.0042, 1.0064, 1.0020, 1.0039, 1.0030]))]), 'divergences': {'chain 0': [4, 18, 38, 42, 58, 68, 72, 75, 78, 81, 83, 146, 225, 274, 443]}, 'acceptance rate': {'chain 0': 0.964}}
